# Daten-Akquise & Sentiment-Engine

***Mathematischer Hintergrund (Meucci 2010, SSRN)**
Nutzung __market-based BL-Version__, dabei wird kein $\tau$ für Estimate Risk verwendet, da Views direkt auf Market X und nicht Params $\mu$.

**Posterior**

$$
\mu_{post} = \pi + \Sigma P^T (P \Sigma P^T + \Omega)^{-1} (v - P \pi)
$$

$$
\Sigma_{post} = \Sigma - \Sigma P^T (P \Sigma P^T + \Omega)^{-1} P \Sigma
$$

Dabei ist $\pi$ aus Equilibrium (inverse Option: $\pi = \lambda \Sigma w_{eq}, \lambda  \approx 2.4$.
P wählt Sektoren (Eye-Matrix -> absolute Views),


**Data Classes (Erweitert für BL-Integration)**

Requirement:

yfinance, newsapi-python, transformers, torch, pandas, numpy, tqdm

In [ ]:
!pip install newsapi-python

In [ ]:
import yfinance
import newsapi  # api_key = " "           #  https://newsapi.org/  kostenloser ApiNews mit email holen
import transformers
import torch
import pandas
import numpy
import tqdm

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils python3.10-venv -y


In [ ]:
import sys
sys.path = [p for p in sys.path if "python3.12" not in p]
sys.path.append("/usr/lib/python3.10/dist-packages")


In [ ]:
!pip install --upgrade --force-reinstall "torch==2.1.2" "torchvision==0.16.2" --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade --force-reinstall "transformers==4.36.2" "accelerate" "sentencepiece"

In [ ]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis", model="ProsusAI/finbert")
nlp("The market reacted positively to the earnings report.")

**Enhanced Code:** Data Acquisition with Logging and Fallbacks

In [ ]:
import logging
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta

import yfinance as yf
import requests
import pandas as pd
import numpy as np
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache

# --- 1. KONFIGURATION ---
NEWS_API_KEY = " "  # Hier deinen Key eintragen .... https://newsapi.org/ kostenloser ApiNews mit email holen
# Preise brauchen Historie für die Kovarianz (60 Tage sind okay bei yfinance)
PRICE_START = (datetime.now() - timedelta(days=60)).strftime('%Y-%m-%d')

SECTORS = {
    "Technology": {"ticker": "XLK", "query": "Apple Microsoft Nvidia"},
    "Energy": {"ticker": "XLE", "query": "Exxon Chevron Oil"},
    "Finance": {"ticker": "XLF", "query": "Goldman Sachs JPMorgan"},
    "Healthcare": {"ticker": "XLV", "query": "Pfizer Johnson Healthcare"}
}

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# --- 2. DATENSTRUKTUREN ---
@dataclass
class SectorPriceData:
    ticker: str
    returns: List[float]

@dataclass
class SectorSentimentScores:
    sector: str
    mean_sentiment: float
    std_sentiment: float
    volume: int

@dataclass
class BlackLittermanInputs:
    pi: np.ndarray
    sigma: np.ndarray
    P: np.ndarray
    v: np.ndarray
    omega: np.ndarray

# --- 3. DIE PIPELINE ---

logging.info("Lade FinBERT (Sentiment Modell)...")
sentiment_pipe = pipeline('sentiment-analysis', model='ProsusAI/finbert')

def fetch_prices(ticker: str) -> SectorPriceData:
    try:
        df = yf.download(ticker, start=PRICE_START, progress=False)
        data = df['Close']
        if isinstance(data, pd.DataFrame): data = data.iloc[:, 0]
        returns = np.log(data / data.shift(1)).dropna().values.flatten().tolist()
        return SectorPriceData(ticker, returns)
    except Exception as e:
        logging.error(f"Fehler bei yfinance ({ticker}): {e}")
        return SectorPriceData(ticker, [])

def fetch_news_free(query: str) -> List[str]:
    """Spezielle Funktion für den NewsAPI Free Plan (kein 'from' Parameter für Historie)."""
    url = "https://newsapi.org/v2/everything"
    # Wir lassen 'from' und 'to' weg, um nur die neuesten erlaubten News zu erhalten
    params = {
        'q': query,
        'apiKey': NEWS_API_KEY,
        'pageSize': 10,
        'language': 'en',
        'sortBy': 'publishedAt' # Aktuellste News zuerst
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        if res.status_code == 426:
            logging.error("Upgrade benötigt für historische Daten. Versuche nur aktuelle News...")
        res.raise_for_status()
        articles = res.json().get('articles', [])
        return [f"{a['title']} {a['description']}" for a in articles]
    except Exception as e:
        logging.error(f"NewsAPI Fehler bei Query '{query}': {e}")
        return []

def compute_sentiments(texts: List[str], sector: str) -> SectorSentimentScores:
    if not texts: return SectorSentimentScores(sector, 0.0, 0.5, 0)
    results = sentiment_pipe([t[:512] for t in texts])
    scores = [r['score'] if r['label'] == 'positive' else -r['score'] if r['label'] == 'negative' else 0 for r in results]
    return SectorSentimentScores(sector, float(np.mean(scores)), float(np.std(scores)) if len(scores) > 1 else 0.5, len(texts))

# --- 4. BLACK-LITTERMAN LOGIK ---



def run_bl_logic(prices: Dict[str, SectorPriceData], sentiments: Dict[str, SectorSentimentScores]):
    sectors = list(prices.keys())
    N = len(sectors)

    # 1. Kovarianz (Sigma) & Markt-Gleichgewicht (Pi)
    ret_df = pd.DataFrame({s: pd.Series(prices[s].returns) for s in sectors}).fillna(0)
    sigma = ret_df.cov().values + np.eye(N) * 1e-6
    pi = 2.4 * (sigma @ (np.ones(N) / N)) # Equilibrium Returns

    # 2. Sentiment Views (v und Omega)
    P = np.eye(N)
    v = np.array([pi[i] + (sentiments[s].mean_sentiment * 0.02) for i, s in enumerate(sectors)])
    omega = np.diag([max((sentiments[s].std_sentiment**2) / max(sentiments[s].volume, 1), 0.0001) for s in sectors])

    # 3. Black-Litterman Formel
    tau = 0.05
    inv_tau_sigma = np.linalg.inv(tau * sigma)
    inv_omega = np.linalg.inv(omega)

    # Posterior Mean (mu_bl)
    # $$\mu_{BL} = [(\tau\Sigma)^{-1} + P^T \Omega^{-1} P]^{-1} [(\tau\Sigma)^{-1} \Pi + P^T \Omega^{-1} v]$$
    mu_bl = np.linalg.inv(inv_tau_sigma + P.T @ inv_omega @ P) @ (inv_tau_sigma @ pi + P.T @ inv_omega @ v)

    # Gewichte berechnen
    w = np.linalg.inv(2.4 * sigma) @ mu_bl
    w = w / np.sum(np.abs(w)) # Normalisierung

    return pi, mu_bl, w

# --- 5. MAIN ---

def main():
    if NEWS_API_KEY == "DEIN_NEWS_API_KEY":
        print("Stopp! Bitte gib erst deinen API-Key ein.")
        return

    price_results, sentiment_results = {}, {}

    # Parallelisierung der Abfragen
    with ThreadPoolExecutor(max_workers=4) as executor:
        p_tasks = {executor.submit(fetch_prices, info['ticker']): s for s, info in SECTORS.items()}
        n_tasks = {executor.submit(fetch_news_free, info['query']): s for s, info in SECTORS.items()}

        for f in as_completed(p_tasks): price_results[p_tasks[f]] = f.result()
        for f in as_completed(n_tasks):
            s = n_tasks[f]
            sentiment_results[s] = compute_sentiments(f.result(), s)

    # Black-Litterman Berechnung
    pi, mu_bl, weights = run_bl_logic(price_results, sentiment_results)

    print("\n" + "="*70)
    print(f"{'SEKTOR':<15} | {'SENTIMENT':<10} | {'EQUIL. RET':<12} | {'BL RET':<12} | {'GEWICHT'}")
    print("-"*70)
    for i, s in enumerate(SECTORS.keys()):
        print(f"{s:<15} | {sentiment_results[s].mean_sentiment:>9.2f} | {pi[i]:>11.2%} | {mu_bl[i]:>11.2%} | {weights[i]:>8.1%}")
    print("="*70)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/tmp/ipython-input-3717581278.py:56: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=PRICE_START, progress=False)



SEKTOR          | SENTIMENT  | EQUIL. RET   | BL RET       | GEWICHT
----------------------------------------------------------------------
Technology      |      0.03 |       0.05% |       0.05% |    25.3%
Energy          |      0.29 |       0.05% |       0.05% |    24.8%
Finance         |      0.23 |       0.05% |       0.05% |    25.0%
Healthcare      |      0.46 |       0.05% |       0.05% |    24.9%


Datenabfrage für Views ohne das Black-Litterman Modell

In [ ]:
import logging
import os
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta

import yfinance as yf
import requests
import pandas as pd
import numpy as np
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache

# --- KONFIGURATION ---
NEWS_API_KEY = " "  # Hier deinen Key von newsapi.org eintragen
START_DATE = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
END_DATE = datetime.now().strftime('%Y-%m-%d')

SECTORS = {
    "Technology": {"ticker": "XLK", "query": "Apple Microsoft Nvidia AI tech"},
    "Energy": {"ticker": "XLE", "query": "Oil Gas Exxon Chevron energy"},
    "Finance": {"ticker": "XLF", "query": "Banking Goldman Sachs JP Morgan interest rates"}
}

# Global logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('pipeline.log'), logging.StreamHandler()]
)

# --- DATA CLASSES ---
@dataclass
class SectorPriceData:
    ticker: str
    dates: List[str]
    closes: List[float]
    returns: List[float]

@dataclass
class SectorNewsItem:
    date: str
    title: str
    description: Optional[str]
    source: str
    url: str

@dataclass
class SectorSentimentScores:
    sector: str
    mean_sentiment: float
    std_sentiment: float
    volume: int

# --- PIPELINE FUNKTIONEN ---

# Sentiment Modell einmalig laden (Lazy Loading)
logging.info("Lade FinBERT Modell (das kann beim ersten Mal kurz dauern)...")
finbert = pipeline('sentiment-analysis', model='ProsusAI/finbert')

@lru_cache(maxsize=128)
def fetch_prices(ticker: str) -> SectorPriceData:
    """Holt Preise via yfinance."""
    try:
        # download liefert ein DataFrame
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)

        if df.empty:
            logging.warning(f"Keine Daten für {ticker} gefunden.")
            return SectorPriceData(ticker, [], [], [])

        # Sicherstellen, dass wir Adj Close oder Close nutzen
        data = df['Close']
        if isinstance(data, pd.DataFrame): # Fallback für Multi-Index
            data = data.iloc[:, 0]

        dates = data.index.strftime('%Y-%m-%d').tolist()
        closes = data.values.tolist()

        # Logarithmische Renditen berechnen
        returns = np.log(data / data.shift(1)).dropna().values.tolist()

        logging.info(f"Preise für {ticker} geladen ({len(closes)} Tage).")
        return SectorPriceData(ticker, dates, closes, returns)
    except Exception as e:
        logging.error(f"yfinance Fehler für {ticker}: {e}")
        return SectorPriceData(ticker, [], [], [])

def fetch_news(sector_name: str, sector_query: str) -> List[SectorNewsItem]:
    """Holt Nachrichten via NewsAPI."""
    url = "https://newsapi.org/v2/everything"
    params = {
        'q': sector_query,
        'from': START_DATE,
        'to': END_DATE,
        'language': 'en',
        'sortBy': 'relevancy',
        'apiKey': NEWS_API_KEY,
        'pageSize': 20
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        articles = response.json().get('articles', [])

        logging.info(f"Gefunden: {len(articles)} News für {sector_name}.")
        return [
            SectorNewsItem(a['publishedAt'], a['title'], a['description'], a['source']['name'], a['url'])
            for a in articles
        ]
    except Exception as e:
        logging.error(f"NewsAPI Fehler für {sector_name}: {e}")
        return []

def compute_sentiments(news_items: List[SectorNewsItem], sector: str) -> SectorSentimentScores:
    """Analysiert die Stimmung der News-Texte."""
    if not news_items:
        return SectorSentimentScores(sector, 0.0, 1.0, 0)

    # Texte für das Modell vorbereiten (Titel + Teaser)
    texts = [f"{item.title} {item.description or ''}"[:512] for item in news_items]
    results = finbert(texts)

    # Mapping: Positive -> 1.0, Negative -> -1.0, Neutral -> 0.0
    scores = []
    for r in results:
        score = r['score']
        if r['label'] == 'negative':
            scores.append(-score)
        elif r['label'] == 'positive':
            scores.append(score)
        else:
            scores.append(0.0)

    scores_np = np.array(scores)
    return SectorSentimentScores(
        sector=sector,
        mean_sentiment=float(np.mean(scores_np)),
        std_sentiment=float(np.std(scores_np)) if len(scores_np) > 1 else 1.0,
        volume=len(news_items)
    )

def run_pipeline(max_workers: int = 4) -> Tuple[Dict[str, SectorPriceData], Dict[str, SectorSentimentScores]]:
    """Führt die Pipeline parallel aus."""
    prices = {}
    sentiments = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Futures für Preise und News erstellen
        price_futures = {executor.submit(fetch_prices, info['ticker']): sector for sector, info in SECTORS.items()}
        news_futures = {executor.submit(fetch_news, sector, info['query']): sector for sector, info in SECTORS.items()}

        # Ergebnisse sammeln
        for future in as_completed(price_futures):
            sector = price_futures[future]
            prices[sector] = future.result()

        for future in as_completed(news_futures):
            sector = news_futures[future]
            news = future.result()
            sentiments[sector] = compute_sentiments(news, sector)

    return prices, sentiments

# --- MAIN ---
if __name__ == "__main__":
    if NEWS_API_KEY == "DEIN_NEWS_API_KEY":
        print("Hinweis: Bitte trage deinen NewsAPI Key ein, um News-Daten zu erhalten.")

    p_data, s_data = run_pipeline()

    print("\n" + "="*40)
    print("PIPELINE RESULTATE")
    print("="*40)
    for sector in SECTORS:
        p = p_data[sector]
        s = s_data[sector]
        last_price = f"{p.closes[-1]:.2f}" if p.closes else "N/A"
        print(f"Sektor: {sector:12} | Preis: {last_price:>7} | Sentiment: {s.mean_sentiment:+.2f} (Vol: {s.volume})")

Device set to use cpu
/tmp/ipython-input-189666731.py:67: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)



PIPELINE RESULTATE
Sektor: Technology   | Preis:  146.08 | Sentiment: -0.09 (Vol: 20)
Sektor: Energy       | Preis:  146.08 | Sentiment: +0.05 (Vol: 20)
Sektor: Finance      | Preis:  146.08 | Sentiment: -0.01 (Vol: 5)


In [ ]:
!pip install newsapi-python

**Flaggship Code**
Black-Litterman Integration Verwende Sentiments für views

In [ ]:
import logging
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import pipeline
import yfinance as yf

# --- 1. KONFIGURATION ---
NEWS_API_KEY = ""  # <--- HIER KEY EINTRAGEN
PRICE_START = (datetime.now() - timedelta(days=60)).strftime('%Y-%m-%d')

SECTORS = {
    "Technology": {"ticker": "XLK", "query": "Apple Microsoft Nvidia AI Semiconductors"},
    "Energy": {"ticker": "XLE", "query": "Oil Gas Exxon Chevron Energy"},
    "Finance": {"ticker": "XLF", "query": "Banking Goldman Sachs JPMorgan Interest Rates"},
    "Healthcare": {"ticker": "XLV", "query": "Pharma Biotech Healthcare"},
    "Consumer": {"ticker": "XLY", "query": "Amazon Tesla Consumer Spending"}
}

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# --- 2. DATENSTRUKTUREN ---
@dataclass
class PortfolioResult:
    sector: str
    ticker: str
    sentiment: float
    news_vol: int
    equilibrium_ret: float
    bl_ret: float
    weight_mkt: float
    weight_bl: float

# --- 3. DIE PIPELINE (Daten & Sentiment) ---

logging.info("Lade FinBERT (Finanz-KI)...")
sentiment_pipe = pipeline('sentiment-analysis', model='ProsusAI/finbert')

def fetch_prices(ticker: str) -> List[float]:
    try:
        df = yf.download(ticker, start=PRICE_START, progress=False)
        data = df['Close']
        if isinstance(data, pd.DataFrame): data = data.iloc[:, 0]
        # Logarithmische Renditen für Normalverteilungs-Annahme
        return np.log(data / data.shift(1)).dropna().values.flatten().tolist()
    except Exception as e:
        logging.error(f"yfinance Fehler ({ticker}): {e}")
        return []

def fetch_news_free(query: str) -> List[str]:
    """Free Tier kompatibel: Keine Zeitstempel-Historie erzwingen."""
    url = "https://newsapi.org/v2/everything"
    params = {'q': query, 'apiKey': NEWS_API_KEY, 'pageSize': 15, 'language': 'en', 'sortBy': 'relevancy'}
    try:
        res = requests.get(url, params=params, timeout=5)
        res.raise_for_status()
        articles = res.json().get('articles', [])
        return [f"{a['title']} {a.get('description', '')}" for a in articles]
    except Exception:
        return []

def analyze_sentiment(texts: List[str]) -> Tuple[float, float, int]:
    if not texts: return 0.0, 0.5, 0
    # FinBERT verarbeitet Texte bis 512 Tokens
    results = sentiment_pipe([t[:512] for t in texts])
    scores = [r['score'] if r['label'] == 'positive' else -r['score'] if r['label'] == 'negative' else 0 for r in results]
    return float(np.mean(scores)), float(np.std(scores)) if len(scores) > 1 else 0.5, len(texts)

# --- 4. MATHEMATIK & VISUALISIERUNG ---



def run_black_litterman(prices: dict, sents: dict):
    sectors = list(prices.keys())
    N = len(sectors)

    # 1. Kovarianz (Sigma) & Markt-Gleichgewicht (Pi)
    ret_df = pd.DataFrame({s: pd.Series(prices[s]) for s in sectors}).fillna(0)
    sigma = ret_df.cov().values + np.eye(N) * 1e-6 # Stabilitäts-Shrinkage

    w_mkt = np.ones(N) / N # Annahme: Gleichgewichtung als Startpunkt
    lambda_risk = 2.4 # Risiko-Aversions-Koeffizient
    pi = lambda_risk * (sigma @ w_mkt)

    # 2. Views aus Sentiment (P, v, Omega)
    P = np.eye(N)
    # Skalierung: 2% Rendite-Einfluss bei vollem Sentiment (1.0)
    v = np.array([pi[i] + (sents[s][0] * 0.02) for i, s in enumerate(sectors)])
    # Omega: Vertrauen in den View basierend auf News-Volumen & Varianz
    omega = np.diag([max((sents[s][1]**2) / max(sents[s][2], 1), 0.0001) for s in sectors])

    # 3. BL Posterior Berechnung
    tau = 0.05
    inv_tau_sigma = np.linalg.inv(tau * sigma)
    inv_omega = np.linalg.inv(omega)

    # Mu_BL (Posterior Mean)
    mu_bl = np.linalg.inv(inv_tau_sigma + P.T @ inv_omega @ P) @ (inv_tau_sigma @ pi + P.T @ inv_omega @ v)

    # 4. Neue Gewichte (Mean-Variance)
    w_bl = np.linalg.inv(lambda_risk * sigma) @ mu_bl
    w_bl = w_bl / np.sum(np.abs(w_bl)) # Normalisierung auf 100%

    return sigma, pi, mu_bl, w_mkt, w_bl

def plot_sentiment_impact(sectors, pi, mu_bl, w_mkt, w_bl, sents):
    """Das neue Dashboard: Gewichtungs-Tilt vs. Rendite-Treiber."""
    fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.12,
                        subplot_titles=("Portfolio-Gewichte (Der Tilt)", "Erwartete Renditen (Der Treiber)"))

    sent_vals = [sents[s][0] for s in sectors]

    # Left: Weights (Markt vs. BL)
    fig.add_trace(go.Bar(x=sectors, y=w_mkt, name='Markt (Prior)', marker_color='lightgray'), row=1, col=1)
    # Dynamische Farben für Gewichte: Blau für Erhöhung, Rot für Senkung
    w_colors = ['#1f77b4' if wb > wm else '#d62728' for wb, wm in zip(w_bl, w_mkt)]
    fig.add_trace(go.Bar(x=sectors, y=w_bl, name='BL (Sentiment)', marker_color=w_colors), row=1, col=1)

    # Right: Returns (Pi vs. Mu_BL)
    fig.add_trace(go.Bar(x=sectors, y=pi, name='Markt-Rendite', marker_color='silver'), row=1, col=2)
    # Dynamische Farben für Rendite: Grün für Positiv-View, Orange für Negativ-View
    r_colors = ['#2ca02c' if sv > 0 else '#ff7f0e' for sv in sent_vals]
    fig.add_trace(go.Bar(x=sectors, y=mu_bl, name='BL-Rendite', marker_color=r_colors), row=1, col=2)

    fig.update_layout(title_text="Portfolio Sentiment Analysis: Black-Litterman Integration",
                      barmode='group', template="plotly_white", height=550)
    fig.update_yaxes(tickformat=".1%", row=1, col=1)
    fig.update_yaxes(tickformat=".2%", row=1, col=2)
    fig.show()

# --- 5. HAUPTPROGRAMM ---

def main():
    if NEWS_API_KEY == "DEIN_NEWS_API_KEY":
        print("Kritischer Fehler: NEWS_API_KEY fehlt!")
        return

    p_results, s_results = {}, {}
    with ThreadPoolExecutor(max_workers=5) as ex:
        p_tasks = {ex.submit(fetch_prices, info['ticker']): s for s, info in SECTORS.items()}
        n_tasks = {ex.submit(fetch_news_free, info['query']): s for s, info in SECTORS.items()}
        for f in as_completed(p_tasks): p_results[p_tasks[f]] = f.result()
        for f in as_completed(n_tasks): s_results[n_tasks[f]] = analyze_sentiment(f.result())

    # BL Berechnung
    sigma, pi, mu_bl, w_mkt, w_bl = run_black_litterman(p_results, s_results)

    # Ergebnis-Objekte erstellen
    final_data = []
    sectors_list = list(SECTORS.keys())
    for i, s in enumerate(sectors_list):
        final_data.append(PortfolioResult(
            s, SECTORS[s]['ticker'], s_results[s][0], s_results[s][2],
            pi[i], mu_bl[i], w_mkt[i], w_bl[i]
        ))

    # Excel Export
    df = pd.DataFrame([vars(r) for r in final_data])
    fname = f"Sentiment_Portfolio_{datetime.now().strftime('%Y%m%d')}.xlsx"
    df.to_excel(fname, index=False)
    logging.info(f"Excel-Report erstellt: {fname}")

    # Visualisierung
    plot_sentiment_impact(sectors_list, pi, mu_bl, w_mkt, w_bl, s_results)

    # Console Output
    print(f"\n{'SEKTOR':<15} | {'SENTIMENT':<10} | {'MKT WEIGHT':<12} | {'BL WEIGHT':<12} | {'DELTA'}")
    print("-" * 75)
    for r in final_data:
        delta = r.weight_bl - r.weight_mkt
        print(f"{r.sector:<15} | {r.sentiment:>9.2f} | {r.weight_mkt:>11.1%} | {r.weight_bl:>11.1%} | {delta:>+7.1%}")

if __name__ == "__main__":
    main()

Device set to use cpu
/tmp/ipython-input-2470123599.py:47: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=PRICE_START, progress=False)



SEKTOR          | SENTIMENT  | MKT WEIGHT   | BL WEIGHT    | DELTA
---------------------------------------------------------------------------
Technology      |     -0.00 |       20.0% |       20.3% |   +0.3%
Energy          |      0.13 |       20.0% |       19.8% |   -0.2%
Finance         |     -0.16 |       20.0% |       20.3% |   +0.3%
Healthcare      |      0.28 |       20.0% |       20.0% |   -0.0%
Consumer        |      0.09 |       20.0% |       19.5% |   -0.5%


**Überblick:**

_Detailliertes Dashboard:_  Plot zeigt links direkt, wie viele Prozentpunkte ein Sektor durch die News gewonnen oder verloren hat (Tilt). Rechts siehst du den "Rendite-Schub" durch das KI-Sentiment.

_Farbleitsystem:_
* Blau/Rot zeigt Veränderungen in der Gewichtung.

* Grün/Orange zeigt positive oder negative Nachrichten-Einflüsse auf die Rendite.

_Excel-Power:_ Die Export-Datei enthält jetzt alle Metriken (Sentiment, Renditen Vorher/Nachher, Gewichte), was ideal für ein Audit oder eine spätere Analyse ist.

_Robustheit:_ Das Skript nutzt asynchrone Threads, um News und Preise gleichzeitig zu laden, was die Laufzeit halbiert.